GloVe의 아이디어를 한 줄로 요약하면 **'임베딩 된 중심 단어와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 만드는 것'**

In [10]:
import pandas as pd
import numpy as np

In [2]:
corpus_path = '../data/word-embeddings/glove/glove.txt'
corpus = [sent.strip().split(" ") for sent in open(corpus_path, 'r', encoding='utf-8').readlines()]

In [3]:
corpus_df = pd.DataFrame(corpus)

In [4]:
corpus_df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,1,-0.427017,-0.543471,0.195273,0.294334,-0.441908,0.586086,-1.443970,-0.128346,-0.419354,...,-0.323161,0.875404,-0.551731,0.462782,0.653174,0.278982,1.287942,0.245155,0.894915,-0.502916
1,.,-0.757692,-1.366592,1.007756,0.140902,-0.405306,-0.137617,-1.532676,-0.855700,-0.578284,...,-0.227633,0.076767,-0.767365,0.378810,1.130215,-0.205914,1.156774,0.879370,0.899059,-0.284559
2,0,-0.224991,-1.100936,0.197688,-0.014826,-1.185218,0.888817,-1.203675,-0.273878,-0.824834,...,-0.479965,1.054976,-0.701015,0.739088,1.177431,0.472935,1.032185,-0.214798,0.706888,-0.081318
3,2,-0.305080,-0.908815,0.179033,0.308061,-1.271097,0.804265,-1.263210,-0.001970,-0.475218,...,-0.508415,1.004855,-0.524278,0.763253,0.693777,0.600829,1.141334,-0.224841,0.649257,-0.208603
4,의,-1.731348,-0.098834,-0.169523,0.454951,-0.729661,0.426901,-1.125615,-0.167185,-0.216998,...,-0.452197,0.722257,0.117114,1.695926,1.328011,1.126812,1.019037,0.701252,0.932216,-0.340602


In [5]:
idx2vocab = dict()
for idx in range(len(corpus_df)):
    idx2vocab[idx] = str(corpus_df.iloc[idx,0])

In [7]:
vocab2idx ={vocab:idx for idx, vocab in idx2vocab.items()}

In [19]:
len(vocab2idx)

358043

In [11]:
ind = corpus_df[0]
corpus_df2 = corpus_df.iloc[:,1:].astype(np.float32).copy()
corpus_df2.index= ind

In [12]:
corpus_df2.head()

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,,,,,,,,,,,,,,,,,,,,,
1,-0.427017,-0.543471,0.195273,0.294334,-0.441908,0.586086,-1.443970,-0.128346,-0.419354,-0.105046,...,-0.323161,0.875404,-0.551731,0.462782,0.653174,0.278982,1.287942,0.245155,0.894915,-0.502916
.,-0.757692,-1.366592,1.007756,0.140902,-0.405306,-0.137617,-1.532676,-0.855700,-0.578284,-0.262581,...,-0.227633,0.076767,-0.767365,0.378810,1.130215,-0.205914,1.156774,0.879370,0.899059,-0.284559
0,-0.224991,-1.100936,0.197688,-0.014826,-1.185218,0.888817,-1.203675,-0.273878,-0.824834,-0.363941,...,-0.479965,1.054976,-0.701015,0.739088,1.177431,0.472935,1.032185,-0.214798,0.706888,-0.081318
2,-0.305080,-0.908815,0.179033,0.308061,-1.271097,0.804265,-1.263210,-0.001970,-0.475218,-0.483079,...,-0.508415,1.004855,-0.524278,0.763253,0.693777,0.600829,1.141334,-0.224841,0.649257,-0.208603
의,-1.731348,-0.098834,-0.169523,0.454951,-0.729661,0.426901,-1.125615,-0.167185,-0.216998,0.428460,...,-0.452197,0.722257,0.117114,1.695926,1.328011,1.126812,1.019037,0.701252,0.932216,-0.340602


In [32]:
import torch
import torch.nn as nn

def similar_words(df, word, k=5):
    cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    
    word_id = vocab2idx[word]
    word_vec = torch.tensor(df.loc[word].values).view(1,-1)
    word_mat = torch.tensor(df.values)
    
    cos_mat = cos(word_vec, word_mat)
    sim, indices = torch.topk(cos_mat, k+1)
    
    word_list = []
    for i in indices:
        if i != word_id:
            i = np.int(i)
            word_list.append(idx2vocab[i])

    return pd.Series(word_list, np.array(sim[1:].detach()))

In [34]:
similar_words(corpus_df2, '희망')

0.759391    행복
0.719031     꿈
0.696173    사랑
0.679551    미래
0.672560    세상
dtype: object

In [35]:
similar_words(corpus_df2, '절망')

0.671587     빠진
0.629165     빠지
0.626621     빠졌
0.613859    빠진다
0.611816    두려움
dtype: object

In [36]:
similar_words(corpus_df2, '학교')

0.916319      초등
0.858939     중학교
0.818967    고등학교
0.762781      교사
0.759017      개교
dtype: object

In [37]:
similar_words(corpus_df2, '학생')

0.746111    대학생
0.741608     교사
0.739201     교육
0.730497     학교
0.711825     입학
dtype: object

In [38]:
similar_words(corpus_df2, '가족')

0.784426    부모
0.750275    부부
0.731567    친구
0.727628    아내
0.721882    결혼
dtype: object

In [39]:
similar_words(corpus_df2, '자동차')

0.719773    트럭
0.708316    차량
0.680790    부품
0.669089    제조
0.667751    전기
dtype: object